### Installing Packages


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv2

# Check TensorFlow version
print(tf.__version__)

In [ ]:
try:
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
except:
    print("TensorFlow setup not working correctly.")

### Preprocess and load data

In [ ]:
import os as os
# Directory paths
person_images_dir = '/kaggle/input/high-resolution-viton-zalando-dataset/test/agnostic-v3.2'
cloth_images_dir = '/kaggle/input/high-resolution-viton-zalando-dataset/test/cloth'
mask_images_dir = '/kaggle/input/high-resolution-viton-zalando-dataset/test/image-parse-v3'
output_images_dir= '/kaggle/input/high-resolution-viton-zalando-dataset/test/image'

# Load all images in the directory
person_images = []
cloth_images = []
mask_images = []
output_images = []

for person_filename, cloth_filename , mask_filename, output_filename in zip(sorted(os.listdir(person_images_dir)), sorted(os.listdir(cloth_images_dir)), sorted (os.listdir(mask_images_dir)), sorted(os.listdir(output_images_dir))):
    person_img_path = os.path.join(person_images_dir, person_filename)
    cloth_img_path = os.path.join(cloth_images_dir, cloth_filename)
    mask_img_path = os.path.join(mask_images_dir, mask_filename)
    output_img_path = os.path.join(output_images_dir, output_filename)
    
    # Load, resize, and normalize the images
    person_image = cv2.imread(person_img_path)
    person_image = cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB)
    person_image = cv2.resize(person_image, (128,128)) / 255.0
    
    cloth_image = cv2.imread(cloth_img_path)
    cloth_image = cv2.cvtColor(cloth_image, cv2.COLOR_BGR2RGB)
    cloth_image = cv2.resize(cloth_image, (128,128)) / 255.0
    
    mask_image = cv2.imread(mask_img_path)
    mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2RGB)
    mask_image = cv2.resize(mask_image, (128,128)) / 255.0
    mask_image = np.mean(mask_image, axis=-1, keepdims=True)
    
    output_image = cv2.imread(output_img_path)
    output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
    output_image = cv2.resize(output_image, (128,128)) / 255.0
    
    person_images.append(person_image)
    cloth_images.append(cloth_image)
    mask_images.append(mask_image)
    output_images.append(output_image)

# Convert to numpy arrays
person_images = np.array(person_images)
cloth_images = np.array(cloth_images)
mask_images = np.array(mask_images)
output_images = np.array(output_images)

# Print the shape of the arrays to verify
print(f"Loaded {person_images.shape[0]} person images.")
print(f"Loaded {cloth_images.shape[0]} cloth images.")
print(f"Loaded {mask_images.shape[0]} mask images.")
print(f"Loaded {output_images.shape[0]} output images.")


In [ ]:
# Verify that the images are loaded correctly
print(f"Person Image shape: {person_image.shape}")
print(f"Cloth Image shape: {cloth_image.shape}")
print(f"Segmentation Image shape: {segmentation_image.shape}")
print(f"Output Image shape: {output_image.shape}")

# Display the loaded images
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title("Person Image")
plt.imshow(segmentation_image)

plt.subplot(1, 2, 2)
plt.title("Cloth Image")
plt.imshow(cloth_image)

plt.show()


### Build the CNN model

In [ ]:
def build_reduced_virtual_tryon_model():
    # Person Image Branch
    person_input = layers.Input(shape=(128, 128, 3))
    x = layers.SeparableConv2D(32, (3, 3), activation='relu', padding='same')(person_input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.SeparableConv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    person_features = layers.Flatten()(x)

    # Cloth Image Branch
    cloth_input = layers.Input(shape=(128, 128, 3))
    y = layers.SeparableConv2D(32, (3, 3), activation='relu', padding='same')(cloth_input)
    y = layers.MaxPooling2D((2, 2))(y)
    y = layers.SeparableConv2D(64, (3, 3), activation='relu', padding='same')(y)
    y = layers.MaxPooling2D((2, 2))(y)
    cloth_features = layers.Flatten()(y)

    # Segmentation Mask Branch
    mask_input = layers.Input(shape=(128, 128, 1))
    z = layers.SeparableConv2D(32, (3, 3), activation='relu', padding='same')(mask_input)
    z = layers.MaxPooling2D((2, 2))(z)
    z = layers.SeparableConv2D(64, (3, 3), activation='relu', padding='same')(z)
    z = layers.MaxPooling2D((2, 2))(z)
    mask_features = layers.Flatten()(z)

    # Merge features
    combined = layers.concatenate([person_features, cloth_features, mask_features])
    w = layers.Dense(512, activation='relu')(combined)
    w = layers.Dense(128 * 128 * 3, activation='sigmoid')(w)
    output_image = layers.Reshape((128, 128, 3))(w)

    # Build and compile the model
    model = models.Model(inputs=[person_input, cloth_input, mask_input], outputs=output_image)
    model.compile(optimizer='adam', loss='mse')
    return model

model_reduced = build_reduced_virtual_tryon_model()
model_reduced.summary()


In [ ]:
# Verify the model structure
print(f"Model has been built with {len(model_reduced.layers)} layers.")


## Training the model

In [ ]:
X_person = np.array(person_images)
X_cloth = np.array(cloth_images)
Y_output = np.array(output_images)

# Now the shape of segmentation_image will be (256, 256, 1)
X_segmentation = np.array(mask_images)

model_reduced.fit([X_person, X_cloth, X_segmentation], Y_output, epochs=20)

In [ ]:
# Verify that the model can run one epoch of training
test_loss = model_reduced.evaluate([X_person, X_cloth, X_segmentation], Y_output)
print(f"Test Loss after one epoch: {test_loss}")


### Visualize Output

In [ ]:
# Test the model on the same input
predicted_image = model_reduced.predict([X_person, X_cloth , X_segmentation])

# Display the original and predicted images
plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.title("Cloth Image")
plt.imshow(X_cloth[0])

plt.subplot(1, 3, 2)
plt.title("Output Image")
plt.imshow(Y_output[0])

plt.subplot(1, 3, 3)
plt.title("Predicted Try-On Image")
plt.imshow(predicted_image[0])

plt.show()


In [ ]:
# from tensorflow.keras.losses import MeanSquaredError

# mse = MeanSquaredError()
# mse_value = mse(Y_output_test, predictions).numpy()
# print(f'MSE: {mse_value}')

def psnr(target, prediction):
    mse = np.mean((target - prediction) ** 2)
    return 10 * np.log10(1.0 / mse)

psnr = psnr(Y_output, predicted_image)
print(psnr)